### scrapy

In [4]:
import scrapy
import requests
from scrapy.http import TextResponse

- gmarket best 200데이터 수집

In [ ]:
# 1. scrapy 프로젝트 생성

In [1]:
!scrapy startproject gmarket

New Scrapy project 'gmarket', using template directory 'c:\users\user\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\day1-20220803T002955Z-001\code\gmarket

You can start your first spider with:
    cd gmarket
    scrapy genspider example example.com


In [2]:
!tree gmarket /f

폴더 PATH의 목록입니다.
볼륨 일련 번호가 00000005 7C6C:FC35입니다.
C:\USERS\USER\DAY1-20220803T002955Z-001\CODE\GMARKET
│  scrapy.cfg
│  
└─gmarket
    │  items.py
    │  middlewares.py
    │  pipelines.py
    │  settings.py
    │  __init__.py
    │  
    └─spiders
            __init__.py
            


In [10]:
# 2. xpath 찾기
url = 'http://corners.gmarket.co.kr/Bestsellers'
request = requests.get(url)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

# links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li[1]/div[1]/a')
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
print(links[:2])

['http://item.gmarket.co.kr/Item?goodscode=2519604129&ver=637953128102552122',
 'http://item.gmarket.co.kr/Item?goodscode=2520466046&ver=637953128102552122']

In [11]:
link = links[0]
request = requests.get(link)
response = TextResponse(request.url, body=request.text, encoding='utf-8')

title = response.xpath('//*[@id="itemcase_basic"]/div/h1/text()')[0].extract()
price = response.xpath('//*[@id="itemcase_basic"]/div/p/span/strong/text()')[0].extract()

print(title, price)

브라운 면도기 세정액 CCR5+1  25,030


In [ ]:
# 3. items.py 코드 작성 | model(데이터 구조) <-- 수집할 데이터의 칼럼 정의

In [17]:
%%writefile gmarket/gmarket/items.py
import scrapy

class GmarketItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    link = scrapy.Field()

Overwriting gmarket/gmarket/items.py


In [ ]:
# 4. spider.py 코드 작성 | crawling 절차 정의

In [18]:
%%writefile gmarket/gmarket/spiders/spider.py
import scrapy
from gmarket.items import GmarketItem

class GMSpider(scrapy.Spider):
    name = "GMB"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response):
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div/ul/li/a/@href').extract()
        for link in links[:20]:
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/h1/text()')[0].extract()
        item["price"] = response.xpath('//*[@id="itemcase_basic"]/div[1]/p/span/strong/text()')[0].extract()
        item["link"] = response.url
        yield item

Overwriting gmarket/gmarket/spiders/spider.py


In [ ]:
# 5. scrapy 프로젝트 실행

In [ ]:
# scrapy crawl GMB -o items.csv # 터미널에서 실행해야함

In [14]:
%pwd

'C:\\Users\\User\\day1-20220803T002955Z-001\\code'

In [19]:
pd.read_csv("gmarket/items.csv")

,link,price,title
0,http://item.gmarket.co.kr/Item?goodscode=22474...,"3,900",10%할인+사은품_책속에서 여름나기 인기도서 260종_무료배송
1,http://item.gmarket.co.kr/Item?goodscode=22825...,"5,900",풀문 여름 시즌오프 버킷햇/레그워머/양말/왕리본/집게핀
2,http://item.gmarket.co.kr/Item?goodscode=12775...,"7,900",무료반품 2+1 균일가 덧신양말남성여성발목중목학생
3,http://item.gmarket.co.kr/Item?goodscode=20396...,"49,900",1+1 2개 무선 선풍기 탁상용 미니 휴대용 캠핑용 벽걸이 사무실 선물용 가정용
4,http://item.gmarket.co.kr/Item?goodscode=97241...,"6,900",22%쿠폰/1+1 2박스외/무료반품/여성팬티/노라인/위생
5,http://item.gmarket.co.kr/Item?goodscode=24572...,"12,000",레고 닌자고 브릭마스터 외 6종 선택구매
6,http://item.gmarket.co.kr/Item?goodscode=25196...,"25,030",브라운 면도기 세정액 CCR5+1
7,http://item.gmarket.co.kr/Item?goodscode=24188...,"16,830",(현대Hmall) 오늘출발 젤리스트로우 투게더팩 1254g X 2팩(총 130개 ...
8,http://item.gmarket.co.kr/Item?goodscode=16359...,"9,900",아삭아삭 미니 오이 20개 /2개주문시 50개발송
9,http://item.gmarket.co.kr/Item?goodscode=20361...,"10,000",무료반품 15% 할인쿠폰 마르조 오나라 단하루특가
